In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("/content/drive/")
!ls

Mounted at /content/drive/
MyDrive  Shareddrives


In [ ]:
# import pandas as pd

# def remove_columns_with_max_except_thread_duration_max(file_path):
#     # Read the CSV file into a pandas DataFrame
#     df = pd.read_csv(file_path)

#     # Get a list of columns to drop
#     columns_to_drop = [col for col in df.columns if "_max" in col or "_avg" in col or "remote" in col and col != "thread_duration_max"]

#         # Print the columns to be removed
#     print("Columns to be removed:")
#     for col in columns_to_drop:
#         print(col)

#     # Drop the columns from the DataFrame
#     df.drop(columns=columns_to_drop, inplace=True)

#     # Save the updated DataFrame back to a new CSV file
#     new_file_path = file_path.replace(".csv", "_updated.csv")
#     df.to_csv(new_file_path, index=False)

#     return new_file_path

# # Replace 'file_path.csv' with the path to your CSV file
# csv_file_path = '/content/drive/MyDrive/Colab Notebooks/data.csv'
# updated_file_path = remove_columns_with_max_except_thread_duration_max(csv_file_path)
# print(f"Columns with '_max' and '_avg' removed, except 'thread_duration_max', in the file '{csv_file_path}' and saved to '{updated_file_path}'.")


In [3]:
import pandas as pd

def remove_columns(file_path):
    # Columns to be removed
    columns_to_drop = [
        "updated_on", "is_deleted", "query_hash", "status", "workspace_id", "workspace",
        "catalog", "cluster_uuid", "planType", "plan", "error", "database", "added_on",
        "email", "operator_id", "table_name", "cluster_name"
    ]

    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)

    # Get a list of columns to drop (with '_max' but not 'thread_duration_max' and additional columns)
    columns_to_drop += [col for col in df.columns if "_max" in col or "_avg" in col or "remote" in col and col != "thread_duration_max"]

    # Print the columns to be removed
    print("Columns to be removed:")
    for col in columns_to_drop:
        print(col)

    # Drop the columns from the DataFrame
    df.drop(columns=columns_to_drop, inplace=True)

    # Save the updated DataFrame back to a new CSV file
    new_file_path = file_path.replace(".csv", "_updated.csv")
    df.to_csv(new_file_path, index=False)

    return new_file_path

# Replace 'file_path.csv' with the path to your CSV file
csv_file_path = '/content/drive/MyDrive/Colab Notebooks/data.csv'
updated_file_path = remove_columns(csv_file_path)
print(f"Columns removed in the file '{csv_file_path}' and saved to '{updated_file_path}'.")


<ipython-input-3-1ff453f30530>:12: DtypeWarning: Columns (11,12,13,14,75,81,87,124,130) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Columns to be removed:
updated_on
is_deleted
query_hash
status
workspace_id
workspace
catalog
cluster_uuid
planType
plan
error
database
added_on
email
operator_id
table_name
cluster_name
file_name_max_read_time
thread_duration_max
RowsInPerThread_max
remoteParallelism
remote_num_chunks
SinkOperator_avg
LogicalValuesOperator_avg
ChunkAppendAndSortOperator_avg
ProjectionOperator_avg
JoinOperator_avg
ColumnarLookupPartBuildOperator_max
InMemoryAggregationOperator_avg
JoinOperator_max
ChunkAppendOperator_avg
ProjectionOperator_max
ColumnarLookupTableBuildOperator_avg
PartInMemoryAggregationOperator_max
TableScanOperator_max
PartInMemoryAggregationOperator_avg
ColumnarLookupPartBuildOperator_avg
TableScanOperator_avg
LimitOperator_avg
InMemoryWindowOperator_avg
InMemoryPartWindowOperator_avg
FilterOperator_avg
InMemoryAggregationOperator_max
InMemoryPartWindowOperator_max
InMemoryWindowOperator_max
FilterOperator_max
ChunkAppendOperator_max
UnionOperator_max
UnionOperator_avg
remote_cost_pe

In [4]:
import pandas as pd

def merge_and_drop_local_columns(file_path):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)

    # Find all columns that have names starting with "local_"
    local_columns = [col for col in df.columns if col.startswith("local_")]

    # Process each local column
    for local_col in local_columns:
        # Get the corresponding non-local column name (removing "local_" prefix)
        non_local_col = local_col.replace("local_", "")

        # Merge data from local column with non-local column where the data is mutually exclusive
        df[non_local_col] = df.apply(lambda row: row[non_local_col] if pd.notnull(row[local_col]) else row[non_local_col], axis=1)

    # Drop the local columns
    df.drop(columns=local_columns, inplace=True)

    # Save the updated DataFrame back to a new CSV file
    new_file_path = file_path.replace(".csv", "_updated.csv")
    df.to_csv(new_file_path, index=False)

    return new_file_path

# Replace 'file_path.csv' with the path to your CSV file
csv_file_path = '/content/drive/MyDrive/Colab Notebooks/data_updated.csv'
updated_file_path = merge_and_drop_local_columns(csv_file_path)
print(f"Data merged and local columns dropped in the file '{csv_file_path}' and saved to '{updated_file_path}'.")


<ipython-input-4-c6836df8ed9f>:5: DtypeWarning: Columns (5,63,70,76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Data merged and local columns dropped in the file '/content/drive/MyDrive/Colab Notebooks/data_updated.csv' and saved to '/content/drive/MyDrive/Colab Notebooks/data_updated_updated.csv'.


We need to check each query ID's sink operator if below query level metrics exist or not, if it exists, then it should reflect on all operators which have the same query id

In [7]:
df = pd.read_csv(csv_file_path)

def validate_data_consistency(df):

    # Group the DataFrame by 'query_id' and apply a custom function to check data consistency
    grouped = df.groupby('query_id')
    for name, group in grouped:
        cols_to_check = ["total_query_time", "queue_blocked_time_Values", "parsingTime",
                         "totalClientQueryTime", "totalOpenDuration", "totalBytes"]

        unique_values = {}
        for col in cols_to_check:
            unique_values[col] = group[col].dropna().unique()

        # Check if more than one unique value exists for any of the specified columns
        if any(len(values) > 1 for values in unique_values.values()):
            raise ValueError(f"Data inconsistency found for query_id: {name}")

        # Fill missing values in the group with the common data for the specified columns
        common_data = {}
        for col, values in unique_values.items():
            if len(values) == 1:
                common_data[col] = values[0]

        df.loc[group.index, cols_to_check] = df.loc[group.index, cols_to_check].fillna(common_data)

    return df

# Assuming you have already read the CSV file into the 'df' DataFrame
# ...

# Merge data from local columns as done previously
# ...

# Validate data consistency for each query_id and fill missing values
df = validate_data_consistency(df)

# Save the updated DataFrame back to a new CSV file
csv_file_path = '/content/drive/MyDrive/Colab Notebooks/data_updated_updated.csv'
new_file_path = csv_file_path.replace(".csv", "_updated.csv")
df.to_csv(new_file_path, index=False)

print(f"Data merged, local columns dropped, and data consistency validated in the file '{csv_file_path}' and saved to '{new_file_path}'.")


<ipython-input-7-499f282bae3f>:1: DtypeWarning: Columns (5,63,70,76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)


Data merged, local columns dropped, and data consistency validated in the file '/content/drive/MyDrive/Colab Notebooks/data_updated_updated.csv' and saved to '/content/drive/MyDrive/Colab Notebooks/data_updated_updated_updated.csv'.
